In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import random
import time

stock_list = [
    'COST', 'UL', 'AMGN', 'UNH', 'WAT', 'UPS', 'LPX', 'WM', 'NVDA',
    'GOOGL', 'MSFT', 'AXP', 'BLK', 'BRK-B', 'NEE', 'XOM', 'CNI'
]

start_date = '2004-10-01'
end_date = '2024-10-01'
train_split = 0.8

# Hyperparameter search spaces
units_options = [16, 32, 64]
layers_options = [1, 2]
lr_options = [0.001, 0.0005]
batch_options = [4, 8]
epoch_options = [50, 100]
n_steps_options = [2, 5, 10]

def load_and_preprocess_data(ticker, n_steps):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)

    if stock_data.empty:
        return None, None, None, None

    target_y = stock_data['Close']
    X_feat = stock_data[['Open', 'High', 'Low']]

    sc_features = StandardScaler()
    sc_target = StandardScaler()

    X_ft_scaled = sc_features.fit_transform(X_feat.values)
    y_scaled = sc_target.fit_transform(target_y.values.reshape(-1, 1)).flatten()

    data = np.concatenate((X_ft_scaled, y_scaled.reshape(-1, 1)), axis=1)
    stock_data_ft = pd.DataFrame(
        data,
        columns=X_feat.columns.tolist() + ['Close_scaled'],
        index=stock_data['Date']
    )

    def lstm_split(data, n_steps):
        X, y = [], []
        for i in range(len(data) - n_steps + 1):
            X.append(data[i:i + n_steps, :-1])
            y.append(data[i + n_steps - 1, -1])
        return np.array(X), np.array(y)

    X1, y1 = lstm_split(stock_data_ft.values, n_steps=n_steps)

    if len(X1) < 20:
        # Not enough data to split meaningfully
        return None, None, None, None

    split_idx = int(np.ceil(len(X1) * train_split))
    X_train, X_test = X1[:split_idx], X1[split_idx:]
    y_train, y_test = y1[:split_idx], y1[split_idx:]

    if len(X_test) < 1:
        return None, None, None, None

    return X_train, X_test, y_train, y_test

def build_lstm_model(n_layers, n_units, learning_rate, input_shape):
    model = Sequential()
    # First LSTM layer
    model.add(LSTM(n_units, input_shape=input_shape, activation='relu', return_sequences=(n_layers > 1)))
    # Additional LSTM layer if specified
    if n_layers > 1:
        model.add(LSTM(n_units, activation='relu'))

    model.add(Dense(1))
    opt = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

def train_and_evaluate_stock(ticker, n_layers, n_units, n_steps, learning_rate, batch_size, epochs):
    X_train, X_test, y_train, y_test = load_and_preprocess_data(ticker, n_steps)
    if X_train is None or X_test is None:
        # Return None to indicate we couldn't train on this stock
        return None

    input_shape = (X_train.shape[1], X_train.shape[2])
    model = build_lstm_model(n_layers, n_units, learning_rate, input_shape=input_shape)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False)

    y_pred_scaled = model.predict(X_test, verbose=0)
    mape = mean_absolute_percentage_error(y_test, y_pred_scaled)
    return mape

def random_search(num_samples=40, print_interval=2):
    best_config = None
    best_score = float('inf')
    start_time = time.time()

    # Store results of all configurations
    all_results = []

    for i in range(num_samples):
        n_units = random.choice(units_options)
        n_layers = random.choice(layers_options)
        lr = random.choice(lr_options)
        batch_size = random.choice(batch_options)
        epochs = random.choice(epoch_options)
        n_steps = random.choice(n_steps_options)

        print(f"\n[INFO] Starting config {i+1}/{num_samples}: Layers={n_layers}, Units={n_units}, n_steps={n_steps}, LR={lr}, Batch={batch_size}, Epochs={epochs}")
        mape_scores = []
        stock_count = 0
        for ticker in stock_list:
            mape = train_and_evaluate_stock(ticker, n_layers, n_units, n_steps, lr, batch_size, epochs)
            stock_count += 1
            if mape is not None:
                mape_scores.append(mape)
            else:
                print(f"[WARN] Could not evaluate {ticker}, skipping.")

            # Print progress every few stocks
            if stock_count % (len(stock_list)//3) == 0:
                elapsed = time.time() - start_time
                print(f"[INFO] Processed {stock_count}/{len(stock_list)} stocks for this config. Time elapsed: {elapsed:.2f}s")

        if len(mape_scores) > 0:
            avg_mape = np.mean(mape_scores)
            print(f"[RESULT] Config {i+1}: Average MAPE={avg_mape:.4f}")
            # Record the results
            all_results.append({
                'Layers': n_layers,
                'Units': n_units,
                'n_steps': n_steps,
                'LR': lr,
                'Batch': batch_size,
                'Epochs': epochs,
                'Avg_MAPE': avg_mape
            })

            if avg_mape < best_score:
                best_score = avg_mape
                best_config = (n_layers, n_units, n_steps, lr, batch_size, epochs)
        else:
            print("[WARN] No valid results for this config.")
            # Still record with None if needed
            all_results.append({
                'Layers': n_layers,
                'Units': n_units,
                'n_steps': n_steps,
                'LR': lr,
                'Batch': batch_size,
                'Epochs': epochs,
                'Avg_MAPE': None
            })

        # Print intermediate best result every few configs
        if (i+1) % print_interval == 0:
            elapsed = time.time() - start_time
            print(f"[INFO] After {i+1} configs, best avg MAPE={best_score:.4f} with config={best_config}. Elapsed time={elapsed:.2f}s")

    # Save all results to CSV
    results_df = pd.DataFrame(all_results)
    results_df.to_csv('hyperparam_results.csv', index=False)
    print("[INFO] All results saved to hyperparam_results.csv")

    return best_config, best_score

if __name__ == "__main__":
    best_config, best_score = random_search(num_samples=10, print_interval=2)
    print("Best Config:", best_config, "with Avg MAPE:", best_score)


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import os

# Configuration chosen from previous best results
n_layers = 1
n_units = 16
n_steps = 5
lr = 0.0005
batch_size = 4
epochs = 100

stock_list = [
    'COST', 'UL', 'AMGN', 'UNH', 'WAT', 'UPS', 'LPX', 'WM', 'NVDA',
    'GOOGL', 'MSFT', 'AXP', 'BLK', 'BRK-B', 'NEE', 'XOM', 'CNI'
]

start_date = '2004-10-01'
end_date = '2024-10-01'
train_split = 0.8

if not os.path.exists("error_analysis_results"):
    os.makedirs("error_analysis_results")

def load_and_preprocess_data(ticker, n_steps):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)

    if stock_data.empty:
        return None, None, None, None, None

    # One-day predictions: we want to predict next day's close
    # Features: Open, High, Low
    # Target: next day's Close
    # Shift Close by -1 day for target
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    # Drop last row due to shift
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    # Prepare features and target
    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day']

    # Scale features and target
    sc_features = StandardScaler()
    sc_target = StandardScaler()

    X_ft_scaled = sc_features.fit_transform(X_feat.values)
    y_scaled = sc_target.fit_transform(y_raw.values.reshape(-1, 1)).flatten()

    data = np.concatenate((X_ft_scaled, y_scaled.reshape(-1, 1)), axis=1)
    # Columns: [Open_scaled, High_scaled, Low_scaled, Close_next_day_scaled]

    def lstm_split(data, n_steps):
        X, y = [], []
        for i in range(len(data) - n_steps + 1):
            X.append(data[i:i + n_steps, :-1])  # all features except last column
            y.append(data[i + n_steps - 1, -1]) # target is at the end of the window
        return np.array(X), np.array(y)

    X1, y1 = lstm_split(data, n_steps=n_steps)

    if len(X1) < 20:
        return None, None, None, None, None

    split_idx = int(np.ceil(len(X1) * train_split))
    X_train, X_test = X1[:split_idx], X1[split_idx:]
    y_train, y_test = y1[:split_idx], y1[split_idx:]

    # Keep track of dates for error analysis
    # The dates correspond to the last day of each n-step window
    # For i-th sample: end date index = i+n_steps-1
    dates = stock_data['Date'].values
    # After LSTM split, the number of samples is len(dates)-n_steps+1
    # The test set dates:
    all_sample_dates = dates[n_steps-1:]
    test_dates = all_sample_dates[split_idx:]

    # We will need scaling inverses for error analysis
    return X_train, X_test, y_train, y_test, (sc_target, test_dates, sc_features)

def build_lstm_model(n_layers, n_units, learning_rate, input_shape):
    model = Sequential()
    model.add(LSTM(n_units, input_shape=input_shape, activation='relu', return_sequences=(n_layers > 1)))
    if n_layers > 1:
        model.add(LSTM(n_units, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

def train_and_get_predictions(ticker):
    # Load data
    data = load_and_preprocess_data(ticker, n_steps)
    if data[0] is None:
        print(f"[WARN] {ticker}: Insufficient data.")
        return None
    X_train, X_test, y_train, y_test, (sc_target, test_dates, sc_features) = data

    input_shape = (X_train.shape[1], X_train.shape[2])
    model = build_lstm_model(n_layers, n_units, lr, input_shape=input_shape)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False)

    y_pred_scaled = model.predict(X_test, verbose=0)

    # Inverse scaling
    y_test_unscaled = sc_target.inverse_transform(y_test.reshape(-1,1)).flatten()
    y_pred_unscaled = sc_target.inverse_transform(y_pred_scaled).flatten()

    # Create a DataFrame for analysis
    df_errors = pd.DataFrame({
        'Date': test_dates,
        'Actual_Close_Next_Day': y_test_unscaled,
        'Predicted_Close_Next_Day': y_pred_unscaled
    })

    df_errors['Absolute_Error'] = np.abs(df_errors['Actual_Close_Next_Day'] - df_errors['Predicted_Close_Next_Day'])
    df_errors['APE'] = df_errors['Absolute_Error'] / np.abs(df_errors['Actual_Close_Next_Day'])  # Absolute Percentage Error
    df_errors['MAPE'] = df_errors['APE'] * 100.0  # percentage

    return df_errors

def analyze_errors_for_all_stocks():
    all_summary = []
    for ticker in stock_list:
        print(f"\n[INFO] Analyzing errors for {ticker}...")
        df_errors = train_and_get_predictions(ticker)
        if df_errors is None or len(df_errors)==0:
            continue

        # Compute overall MAPE
        overall_mape = df_errors['MAPE'].mean()
        print(f"[RESULT] {ticker} Overall MAPE: {overall_mape:.2f}%")

        # Sort by MAPE (highest errors first)
        df_sorted = df_errors.sort_values('MAPE', ascending=False)

        # Print top 10 worst predictions
        worst_10 = df_sorted.head(10)
        print(f"[INFO] Top 10 days with highest error for {ticker}:")
        print(worst_10[['Date', 'Actual_Close_Next_Day', 'Predicted_Close_Next_Day', 'MAPE']])

        # Save full error DataFrame to CSV
        df_errors.to_csv(f"error_analysis_results/{ticker}_error_analysis.csv", index=False)

        # Store summary
        all_summary.append({
            'Ticker': ticker,
            'Overall_MAPE': overall_mape
        })

    # Overall summary
    summary_df = pd.DataFrame(all_summary)
    summary_df.to_csv("error_analysis_results/summary.csv", index=False)
    print("\n[INFO] Error analysis completed. Summary saved to error_analysis_results/summary.csv")

if __name__ == "__main__":
    analyze_errors_for_all_stocks()


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import os
import math
import time

# Configuration chosen from previous best results (example)
n_layers = 1
n_units = 16
n_steps = 5
lr = 0.0005
batch_size = 4
epochs = 50  # Consider fewer epochs for repeated training

# We will only test on Costco
ticker = 'COST'

# Reduced date range
start_date = '2020-01-01'
end_date = '2024-01-01'

if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")

def load_and_preprocess_data(ticker, n_steps=5):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None, None, None, None

    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day']

    sc_features = StandardScaler()
    sc_target = StandardScaler()

    X_ft_scaled = sc_features.fit_transform(X_feat.values)
    y_scaled = sc_target.fit_transform(y_raw.values.reshape(-1, 1)).flatten()

    data = np.concatenate((X_ft_scaled, y_scaled.reshape(-1,1)), axis=1)
    dates = stock_data['Date'].values

    def lstm_split(data, n_steps):
        X, y = [], []
        for i in range(len(data) - n_steps + 1):
            X.append(data[i:i+n_steps, :-1])
            y.append(data[i+n_steps-1, -1])
        return np.array(X), np.array(y)

    X_all, y_all = lstm_split(data, n_steps)

    if len(X_all) < 20:
        return None, None, None, None, None, None

    sample_end_dates = dates[n_steps-1:]
    return X_all, y_all, sample_end_dates, sc_target, sc_features, stock_data

def build_lstm_model(n_layers, n_units, learning_rate, input_shape):
    model = Sequential()
    model.add(LSTM(n_units, activation='relu', return_sequences=(n_layers > 1), input_shape=input_shape))
    if n_layers > 1:
        model.add(LSTM(n_units, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

def walk_forward_evaluation(X_all, y_all, sample_end_dates, sc_target, window_size=200):
    total_samples = len(X_all)
    if total_samples <= window_size:
        return None

    predictions = []
    actuals = []
    prediction_dates = []

    start_time = time.time()
    progress_interval = 50

    for i in range(total_samples - window_size):
        X_train = X_all[i:i+window_size]
        y_train = y_all[i:i+window_size]
        X_test = X_all[i+window_size:i+window_size+1]
        y_test = y_all[i+window_size:i+window_size+1]

        model = build_lstm_model(n_layers, n_units, lr, input_shape=(X_train.shape[1], X_train.shape[2]))
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False)

        y_pred_scaled = model.predict(X_test, verbose=0)
        y_test_unscaled = sc_target.inverse_transform(y_test.reshape(-1,1)).flatten()
        y_pred_unscaled = sc_target.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        prediction_dates.append(sample_end_dates[i+window_size])

        if (i+1) % progress_interval == 0:
            elapsed = time.time() - start_time
            print(f"[INFO] Processed {i+1}/{total_samples - window_size} predictions for {ticker}. Elapsed: {elapsed:.2f}s")

    df_results = pd.DataFrame({
        'Date': prediction_dates,
        'Actual_Close_Next_Day': actuals,
        'Predicted_Close_Next_Day': predictions
    })

    df_results['Absolute_Error'] = np.abs(df_results['Actual_Close_Next_Day'] - df_results['Predicted_Close_Next_Day'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual_Close_Next_Day'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    return df_results

def analyze_rolling_for_costco(window_size=200):
    print(f"[INFO] Rolling evaluation for {ticker} (2020-2024 data)...")
    data = load_and_preprocess_data(ticker, n_steps)
    if data is None or data[0] is None:
        print(f"[WARN] {ticker}: Not enough data.")
        return
    X_all, y_all, sample_end_dates, sc_target, sc_features, stock_data = data
    df_results = walk_forward_evaluation(X_all, y_all, sample_end_dates, sc_target, window_size=window_size)
    if df_results is None or len(df_results)==0:
        print(f"[WARN] {ticker}: Not enough samples for rolling evaluation.")
        return

    overall_mape = df_results['MAPE'].mean()
    print(f"[RESULT] {ticker} Overall MAPE (Rolling): {overall_mape:.2f}%")

    worst_10 = df_results.sort_values('MAPE', ascending=False).head(10)
    print("[INFO] Top 10 worst predictions for COST:")
    print(worst_10[['Date', 'Actual_Close_Next_Day', 'Predicted_Close_Next_Day', 'MAPE']])

    df_results.to_csv(f"rolling_error_analysis_results/{ticker}_rolling_results.csv", index=False)
    print("[INFO] Rolling error analysis completed for COST. Results saved.")

if __name__ == "__main__":
    analyze_rolling_for_costco(window_size=200)


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import os
import math
import time

########################
# Configuration
########################
ticker = 'COST'
start_date = '2020-01-01'
end_date = '2024-01-01'

n_steps = 5
window_size = 200
step_size = 10
initial_epochs = 50
update_epochs = 20

initial_lr = 0.0005
update_lr = 0.0002

train_ratio = 0.9

if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")

########################
# Data Loading Functions
########################
def load_full_data_helper(ticker):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None

    return X_feat.values, y_raw, dates

def load_full_data(ticker):
    X_feat, y_raw, dates = load_full_data_helper(ticker)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates

########################
# Model Creation
########################
def create_lstm_model(n_layers, n_units, learning_rate, input_shape):
    model = Sequential()
    model.add(LSTM(n_units, activation='relu', return_sequences=(n_layers > 1), input_shape=input_shape))
    if n_layers > 1:
        model.add(LSTM(n_units, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

########################
# Window Extraction
########################
def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    # Create samples before scaling to determine sample count
    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    # Scaling with RobustScaler
    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()

    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)

    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()

    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)

########################
# Train on Window
########################
def train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=None):
    total_samples = len(X_samples)
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    input_shape = (X_train.shape[1], X_train.shape[2])
    model = create_lstm_model(1, 16, lr, input_shape)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    model.fit(X_train, y_train, epochs=epochs, batch_size=4, verbose=0, shuffle=False, validation_data=(X_val, y_val))

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape

########################
# Rolling Training
########################
def rolling_training(ticker, window_size=200, step_size=10, n_steps=5):
    data = load_full_data(ticker)
    if data[0] is None:
        print("[WARN] Not enough data.")
        return
    X_full, y_full, dates_full = data

    start_time = time.time()

    # Initial window
    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print("[WARN] Initial window not valid.")
        return
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    print("[INFO] Training initial window...")
    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=True, prev_weights=None)
    print(f"[RESULT] Initial window validation MAPE: {val_mape:.2f}%")

    weights_path = "rolling_error_analysis_results/cost_initial.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print("[INFO] No more windows can be extracted.")
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: training on new window starting at {start_idx}")
        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=weights_path)
        print(f"[RESULT] Window validation MAPE: {val_mape:.2f}%")

        weights_path = f"rolling_error_analysis_results/cost_iteration_{iteration}.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] Predicted last sample of window: Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv("rolling_error_analysis_results/cost_rolling_results.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training completed for COST. Overall MAPE across predictions: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] Total elapsed time: {elapsed:.2f}s")

    worst_10 = df_results.sort_values('MAPE', ascending=False).head(10)
    print("[INFO] Top 10 worst predictions in rolling scenario:")
    print(worst_10[['Date', 'Actual', 'Predicted', 'MAPE']])

if __name__ == "__main__":
    rolling_training(ticker, window_size=200, step_size=10, n_steps=5)


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import os
import math
import time
import matplotlib.pyplot as plt

########################
# Configuration
########################
ticker = 'COST'
start_date = '2020-01-01'
end_date = '2024-01-01'

n_steps = 5
window_size = 200
step_size = 10
initial_epochs = 50
update_epochs = 20

initial_lr = 0.0005
update_lr = 0.0002

train_ratio = 0.9

if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")

def load_full_data_helper(ticker):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates

def load_full_data(ticker):
    X_feat, y_raw, dates = load_full_data_helper(ticker)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates

def create_lstm_model(n_layers, n_units, learning_rate, input_shape):
    model = Sequential()
    model.add(LSTM(n_units, activation='relu', return_sequences=(n_layers > 1), input_shape=input_shape))
    if n_layers > 1:
        model.add(LSTM(n_units, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)
    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)

def train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=None):
    total_samples = len(X_samples)
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    input_shape = (X_train.shape[1], X_train.shape[2])
    model = create_lstm_model(1, 16, lr, input_shape)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    # No early stopping here, run all epochs
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=4, verbose=0, shuffle=False,
                        validation_data=(X_val, y_val))

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape, history

def rolling_training_no_early_stopping_worst10(ticker, window_size=200, step_size=10, n_steps=5):
    X_full, y_full, dates_full = load_full_data(ticker)
    if X_full is None:
        print("[WARN] Not enough data.")
        return

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print("[WARN] Initial window not valid.")
        return
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    print("[INFO] Training initial window (no early stopping)...")
    model, val_mape, history_init = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=True, prev_weights=None)
    print(f"[RESULT] Initial window validation MAPE: {val_mape:.2f}%")

    weights_path = "rolling_error_analysis_results/cost_initial_noes.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []
    histories = [(0, history_init)]  # 0 for initial

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print("[INFO] No more windows can be extracted.")
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: training on new window (no early stopping)...")
        model, val_mape, history_iter = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=weights_path)
        print(f"[RESULT] Window validation MAPE: {val_mape:.2f}%")
        histories.append((iteration, history_iter))

        weights_path = f"rolling_error_analysis_results/cost_iteration_{iteration}_noes.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] Predicted last sample of window: Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv("rolling_error_analysis_results/cost_rolling_results_noes.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training (no early stopping) completed for COST. Overall MAPE: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] Total elapsed time: {elapsed:.2f}s")

    worst_10 = df_results.sort_values('MAPE', ascending=False).head(10)
    print("[INFO] Top 10 worst predictions in rolling scenario (No Early Stopping):")
    print(worst_10[['Date', 'Actual', 'Predicted', 'MAPE']])

    # Extract iterations from worst_10 predictions
    # iteration index starts at 1 for first update, df_results starts at 0
    worst_iterations = [i+1 for i in worst_10.index]

    # Filter histories to only include these worst iterations
    filtered_histories = [(it, hist) for (it, hist) in histories if it in worst_iterations]

    # Plot only these worst iterations
    plt.figure(figsize=(10,6))
    for (it, hist) in filtered_histories:
        train_loss = hist.history['loss']
        val_loss = hist.history['val_loss']
        epochs_range = range(1, len(train_loss)+1)
        plt.plot(epochs_range, train_loss, label=f'Iter_{it}_train_loss')
        plt.plot(epochs_range, val_loss, label=f'Iter_{it}_val_loss')

    plt.title("Training and Validation Loss for 10 Worst MAPE Iterations (No Early Stopping)")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.savefig("rolling_error_analysis_results/worst_10_training_curves_noes.png")
    print("[INFO] Training curves for 10 worst MAPE iterations (No Early Stopping) saved as worst_10_training_curves_noes.png")

if __name__ == "__main__":
    rolling_training_no_early_stopping_worst10(ticker, window_size=200, step_size=10, n_steps=5)


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration
########################
ticker = 'COST'
start_date = '2020-01-01'
end_date = '2024-01-01'

n_steps = 5
window_size = 200
step_size = 10
initial_epochs = 100
update_epochs = 100

initial_lr = 0.0005
update_lr = 0.0002

train_ratio = 0.9

if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")

def load_full_data_helper(ticker):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates

def load_full_data(ticker):
    X_feat, y_raw, dates = load_full_data_helper(ticker)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates

def create_lstm_model(n_layers, n_units, learning_rate, input_shape):
    model = Sequential()
    model.add(LSTM(n_units, activation='relu', return_sequences=(n_layers > 1), input_shape=input_shape))
    if n_layers > 1:
        model.add(LSTM(n_units, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)
    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)

def train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=None):
    total_samples = len(X_samples)
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    input_shape = (X_train.shape[1], X_train.shape[2])
    model = create_lstm_model(1, 16, lr, input_shape)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    # EarlyStopping callback
    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=4, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape

def rolling_training_early_stopping(ticker, window_size=200, step_size=10, n_steps=5):
    X_full, y_full, dates_full = load_full_data(ticker)
    if X_full is None:
        print("[WARN] Not enough data.")
        return

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print("[WARN] Initial window not valid.")
        return
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    print("[INFO] Training initial window with Early Stopping...")
    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=True, prev_weights=None)
    print(f"[RESULT] Initial window validation MAPE: {val_mape:.2f}%")

    weights_path = "rolling_error_analysis_results/cost_initial_es.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print("[INFO] No more windows can be extracted.")
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: training on new window with Early Stopping...")
        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=weights_path)
        print(f"[RESULT] Window validation MAPE: {val_mape:.2f}%")

        weights_path = f"rolling_error_analysis_results/cost_iteration_{iteration}_es.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] Predicted last sample of window: Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv("rolling_error_analysis_results/cost_rolling_results_es.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training with Early Stopping completed for COST. Overall MAPE: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] Total elapsed time: {elapsed:.2f}s")

    worst_10 = df_results.sort_values('MAPE', ascending=False).head(10)
    print("[INFO] Top 10 worst predictions in rolling scenario (Early Stopping):")
    print(worst_10[['Date', 'Actual', 'Predicted', 'MAPE']])

if __name__ == "__main__":
    rolling_training_early_stopping(ticker, window_size=200, step_size=10, n_steps=5)


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import os
import math
import time

########################
# Configuration
########################
ticker = 'COST'
start_date = '2020-01-01'
end_date = '2024-01-01'

n_steps = 5
window_size = 200
step_size = 10
initial_epochs = 50
update_epochs = 20
initial_lr = 0.0005
update_lr = 0.0002
train_ratio = 0.9
dropout_rate = 0.01

if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")

def load_full_data_helper(ticker):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates

def load_full_data(ticker):
    X_feat, y_raw, dates = load_full_data_helper(ticker)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates

from tensorflow.keras.layers import LSTM, Dense, Dropout

def create_lstm_model(n_layers, n_units, learning_rate, input_shape, dropout=0.2):
    model = Sequential()
    model.add(LSTM(n_units, activation='relu', return_sequences=(n_layers > 1),
                   input_shape=input_shape, dropout=dropout, recurrent_dropout=dropout))
    if n_layers > 1:
        model.add(LSTM(n_units, activation='relu', dropout=dropout, recurrent_dropout=dropout))
    model.add(Dense(1))
    opt = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None
    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    from sklearn.preprocessing import RobustScaler
    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)
    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)

def train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=None, dropout=0.2):
    total_samples = len(X_samples)
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    input_shape = (X_train.shape[1], X_train.shape[2])
    model = create_lstm_model(1, 16, lr, input_shape, dropout=dropout)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    model.fit(X_train, y_train, epochs=epochs, batch_size=4, verbose=0, shuffle=False, validation_data=(X_val, y_val))

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape

def rolling_training_dropout(ticker, window_size=200, step_size=10, n_steps=5, dropout=0.2):
    X_full, y_full, dates_full = load_full_data(ticker)
    if X_full is None:
        print("[WARN] Not enough data.")
        return

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print("[WARN] Initial window not valid.")
        return
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    print("[INFO] Training initial window with Dropout...")
    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=True, prev_weights=None, dropout=dropout)
    print(f"[RESULT] Initial window validation MAPE: {val_mape:.2f}%")

    weights_path = "rolling_error_analysis_results/cost_initial_dropout.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print("[INFO] No more windows can be extracted.")
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: training on new window with Dropout...")
        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=weights_path, dropout=dropout)
        print(f"[RESULT] Window validation MAPE: {val_mape:.2f}%")

        weights_path = f"rolling_error_analysis_results/cost_iteration_{iteration}_dropout.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] Predicted last sample of window: Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv("rolling_error_analysis_results/cost_rolling_results_dropout.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training with Dropout completed for COST. Overall MAPE: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] Total elapsed time: {elapsed:.2f}s")

    worst_10 = df_results.sort_values('MAPE', ascending=False).head(10)
    print("[INFO] Top 10 worst predictions in rolling scenario (Dropout):")
    print(worst_10[['Date', 'Actual', 'Predicted', 'MAPE']])

if __name__ == "__main__":
    rolling_training_dropout(ticker, window_size=200, step_size=10, n_steps=5, dropout=0.2)


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration
########################
ticker = 'COST'
start_date = '2020-01-01'
end_date = '2024-01-01'

n_steps = 5
window_size = 200
step_size = 10
initial_epochs = 100
update_epochs = 100

initial_lr = 0.0005
update_lr = 0.0002

train_ratio = 0.9

if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")

def load_full_data_helper(ticker):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates

def load_full_data(ticker):
    X_feat, y_raw, dates = load_full_data_helper(ticker)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates

def create_lstm_model(n_layers, n_units, learning_rate, input_shape):
    model = Sequential()
    model.add(LSTM(n_units, activation='relu', return_sequences=(n_layers > 1), input_shape=input_shape))
    if n_layers > 1:
        model.add(LSTM(n_units, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

def apply_scaling(X_window, y_window, method):
    """
    Apply chosen scaling method to X_window and y_window.
    method can be 'robust', 'log', or 'rolling_stats'.
    Returns X_scaled, y_scaled, X_transform_info, y_transform_info
    where transform_info can be used for inverse transformation later.
    """
    if method == 'robust':
        # Use RobustScaler as originally
        feat_scaler = RobustScaler()
        target_scaler = RobustScaler()
        X_flat = X_window.reshape(len(X_window), -1)
        feat_scaler.fit(X_flat)
        y_window_reshaped = y_window.reshape(-1,1)
        target_scaler.fit(y_window_reshaped)
        X_scaled_flat = feat_scaler.transform(X_flat)
        y_scaled = target_scaler.transform(y_window_reshaped).flatten()
        X_scaled = X_scaled_flat.reshape(X_window.shape)
        # Store scalers for inverse
        return X_scaled, y_scaled, ('robust', feat_scaler), ('robust', target_scaler)

    elif method == 'log':
        # Log transform features and target, then standard scale
        # Add a small constant to avoid log(0) if necessary
        epsilon = 1e-3
        X_log = np.log(X_window + epsilon)
        y_log = np.log(y_window.reshape(-1,1) + epsilon)

        feat_scaler = StandardScaler()
        target_scaler = StandardScaler()
        X_flat = X_log.reshape(len(X_window), -1)
        feat_scaler.fit(X_flat)
        target_scaler.fit(y_log)
        X_scaled_flat = feat_scaler.transform(X_flat)
        y_scaled = target_scaler.transform(y_log).flatten()
        X_scaled = X_scaled_flat.reshape(X_window.shape)
        # Store info that we used log transform and epsilon
        return X_scaled, y_scaled, ('log', feat_scaler, epsilon), ('log', target_scaler, epsilon)

    elif method == 'rolling_stats':
        # Compute median and IQR for each feature in X_window and for target
        # We'll apply median/IQR scaling similar to a robust scaling but manually
        X_median = np.median(X_window, axis=0)
        X_q1 = np.percentile(X_window, 25, axis=0)
        X_q3 = np.percentile(X_window, 75, axis=0)
        X_iqr = X_q3 - X_q1
        # Avoid divide by zero
        X_iqr[X_iqr == 0] = 1e-3
        X_scaled = (X_window - X_median) / X_iqr

        y_median = np.median(y_window)
        y_q1 = np.percentile(y_window, 25)
        y_q3 = np.percentile(y_window, 75)
        y_iqr = y_q3 - y_q1
        if y_iqr == 0:
            y_iqr = 1e-3
        y_scaled = (y_window - y_median) / y_iqr
        return X_scaled, y_scaled, ('rolling_stats', X_median, X_iqr), ('rolling_stats', y_median, y_iqr)

def lstm_split(dataX, dataY, n_steps):
    X, y = [], []
    for i in range(len(dataX)-n_steps+1):
        X.append(dataX[i:i+n_steps])
        y.append(dataY[i+n_steps-1])
    return np.array(X), np.array(y)

def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps, scaling_method='robust'):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    # We first scale X_window and y_window
    X_scaled, y_scaled, X_info, y_info = apply_scaling(X_window, y_window, scaling_method)

    X_samples, y_samples = lstm_split(X_scaled, y_scaled, n_steps)
    sample_dates = dates_window[n_steps-1:]

    return X_samples, y_samples, sample_dates, X_info, y_info, (X_window, y_window, dates_window)

def inverse_transform(y_pred_scaled, y_true_scaled, y_info):
    method = y_info[0]
    if method == 'robust':
        target_scaler = y_info[1]
        # inverse_transform expects a 2D array
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled.reshape(-1,1)).flatten()
        y_true_unscaled = target_scaler.inverse_transform(y_true_scaled.reshape(-1,1)).flatten()
        return y_pred_unscaled, y_true_unscaled

    elif method == 'log':
        target_scaler = y_info[1]
        epsilon = y_info[2]
        y_pred_log = target_scaler.inverse_transform(y_pred_scaled.reshape(-1,1))
        y_true_log = target_scaler.inverse_transform(y_true_scaled.reshape(-1,1))
        # Now exponentiate and subtract epsilon
        y_pred_unscaled = np.exp(y_pred_log) - epsilon
        y_true_unscaled = np.exp(y_true_log) - epsilon
        return y_pred_unscaled.flatten(), y_true_unscaled.flatten()

    elif method == 'rolling_stats':
        y_median = y_info[1]
        y_iqr = y_info[2]
        y_pred_unscaled = (y_pred_scaled * y_iqr) + y_median
        y_true_unscaled = (y_true_scaled * y_iqr) + y_median
        return y_pred_unscaled, y_true_unscaled

def train_on_window(X_samples, y_samples, X_info, y_info, initial_training=False, prev_weights=None):
    total_samples = len(X_samples)
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    input_shape = (X_train.shape[1], X_train.shape[2])
    model = create_lstm_model(1, 16, lr, input_shape)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=4, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    # Compute validation MAPE
    y_val_pred_scaled = model.predict(X_val, verbose=0)
    # For MAPE, we need inverse transform
    y_val_pred_unscaled, y_val_unscaled = inverse_transform(y_val_pred_scaled.flatten(), y_val.flatten(), y_info)
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape

def rolling_training_early_stopping(ticker, window_size=200, step_size=10, n_steps=5, scaling_method='robust'):
    X_full, y_full, dates_full = load_full_data(ticker)
    if X_full is None:
        print("[WARN] Not enough data.")
        return

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps, scaling_method)
    if window_data[0] is None:
        print("[WARN] Initial window not valid.")
        return
    X_samples, y_samples, sample_dates, X_info, y_info, (Xw, yw, dw) = window_data

    print(f"[INFO] Training initial window with Early Stopping using {scaling_method} scaling...")
    model, val_mape = train_on_window(X_samples, y_samples, X_info, y_info, initial_training=True, prev_weights=None)
    print(f"[RESULT] Initial window validation MAPE ({scaling_method}): {val_mape:.2f}%")

    weights_path = f"rolling_error_analysis_results/cost_initial_es_{scaling_method}.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []
    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps, scaling_method)
        if window_data[0] is None:
            print("[INFO] No more windows can be extracted.")
            break
        X_samples, y_samples, sample_dates, X_info, y_info, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: training on new window with Early Stopping ({scaling_method})...")
        model, val_mape = train_on_window(X_samples, y_samples, X_info, y_info, initial_training=False, prev_weights=weights_path)
        print(f"[RESULT] Window validation MAPE ({scaling_method}): {val_mape:.2f}%")

        weights_path = f"rolling_error_analysis_results/cost_iteration_{iteration}_es_{scaling_method}.weights.h5"
        model.save_weights(weights_path)

        # Predict last sample
        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_pred_unscaled, y_test_unscaled = inverse_transform(y_pred_scaled.flatten(), y_test_last.flatten(), y_info)
        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] Predicted last sample of window ({scaling_method}): Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv(f"rolling_error_analysis_results/cost_rolling_results_es_{scaling_method}.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training with Early Stopping completed for COST using {scaling_method} scaling. Overall MAPE: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] Total elapsed time: {elapsed:.2f}s")

    # Print worst 10 predictions
    worst_10 = df_results.sort_values('MAPE', ascending=False).head(10)
    print(f"[INFO] Top 10 worst predictions in rolling scenario (Early Stopping, {scaling_method}):")
    print(worst_10[['Date', 'Actual', 'Predicted', 'MAPE']])

if __name__ == "__main__":
    # Compare three scaling methods: 'robust', 'log', 'rolling_stats'
    for method in ['robust', 'log', 'rolling_stats']:
        rolling_training_early_stopping(ticker, window_size=200, step_size=10, n_steps=5, scaling_method=method)


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration
########################
ticker = 'COST'
start_date = '2020-01-01'
end_date = '2024-01-01'

n_steps = 5
window_size = 200
step_size = 10
initial_epochs = 100
update_epochs = 100

initial_lr = 0.0005
update_lr = 0.0002

train_ratio = 0.9

if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")

def load_full_data_helper(ticker):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates

def load_full_data(ticker):
    X_feat, y_raw, dates = load_full_data_helper(ticker)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates

def create_lstm_model(n_layers, n_units, learning_rate, input_shape):
    model = Sequential()
    model.add(LSTM(n_units, activation='relu', return_sequences=(n_layers > 1), input_shape=input_shape))
    if n_layers > 1:
        model.add(LSTM(n_units, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)
    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)

def train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=None):
    total_samples = len(X_samples)
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    input_shape = (X_train.shape[1], X_train.shape[2])
    model = create_lstm_model(1, 16, lr, input_shape)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    # EarlyStopping callback
    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=4, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape

def rolling_training_early_stopping(ticker, window_size=200, step_size=10, n_steps=5):
    X_full, y_full, dates_full = load_full_data(ticker)
    if X_full is None:
        print("[WARN] Not enough data.")
        return

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print("[WARN] Initial window not valid.")
        return
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    print("[INFO] Training initial window with Early Stopping...")
    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=True, prev_weights=None)
    print(f"[RESULT] Initial window validation MAPE: {val_mape:.2f}%")

    weights_path = "rolling_error_analysis_results/cost_initial_es.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print("[INFO] No more windows can be extracted.")
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: training on new window with Early Stopping...")
        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=weights_path)
        print(f"[RESULT] Window validation MAPE: {val_mape:.2f}%")

        weights_path = f"rolling_error_analysis_results/cost_iteration_{iteration}_es.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] Predicted last sample of window: Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv("rolling_error_analysis_results/cost_rolling_results_es.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training with Early Stopping completed for COST. Overall MAPE: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] Total elapsed time: {elapsed:.2f}s")

    worst_10 = df_results.sort_values('MAPE', ascending=False).head(10)
    print("[INFO] Top 10 worst predictions in rolling scenario (Early Stopping):")
    print(worst_10[['Date', 'Actual', 'Predicted', 'MAPE']])

if __name__ == "__main__":
    rolling_training_early_stopping(ticker, window_size=200, step_size=10, n_steps=5)


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration
########################
ticker = 'COST'
start_date = '2020-01-01'
end_date = '2024-01-01'

n_steps = 10
window_size = 200
step_size = 10
initial_epochs = 100
update_epochs = 100

initial_lr = 0.0005
update_lr = 0.0002

train_ratio = 0.9

if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")

def load_full_data_helper(ticker):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates

def load_full_data(ticker):
    X_feat, y_raw, dates = load_full_data_helper(ticker)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates

def create_lstm_model(n_layers, n_units, learning_rate, input_shape):
    model = Sequential()
    model.add(LSTM(n_units, activation='relu', return_sequences=(n_layers > 1), input_shape=input_shape))
    if n_layers > 1:
        model.add(LSTM(n_units, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)
    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)

def train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=None):
    total_samples = len(X_samples)
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    input_shape = (X_train.shape[1], X_train.shape[2])
    model = create_lstm_model(1, 16, lr, input_shape)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    # EarlyStopping callback
    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=4, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape

def rolling_training_early_stopping(ticker, window_size=200, step_size=10, n_steps=10):
    X_full, y_full, dates_full = load_full_data(ticker)
    if X_full is None:
        print("[WARN] Not enough data.")
        return

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print("[WARN] Initial window not valid.")
        return
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    print("[INFO] Training initial window with Early Stopping...")
    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=True, prev_weights=None)
    print(f"[RESULT] Initial window validation MAPE: {val_mape:.2f}%")

    weights_path = "rolling_error_analysis_results/cost_initial_es.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print("[INFO] No more windows can be extracted.")
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: training on new window with Early Stopping...")
        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=weights_path)
        print(f"[RESULT] Window validation MAPE: {val_mape:.2f}%")

        weights_path = f"rolling_error_analysis_results/cost_iteration_{iteration}_es.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] Predicted last sample of window: Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv("rolling_error_analysis_results/cost_rolling_results_es.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training with Early Stopping completed for COST. Overall MAPE: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] Total elapsed time: {elapsed:.2f}s")

    worst_10 = df_results.sort_values('MAPE', ascending=False).head(10)
    print("[INFO] Top 10 worst predictions in rolling scenario (Early Stopping):")
    print(worst_10[['Date', 'Actual', 'Predicted', 'MAPE']])

if __name__ == "__main__":
    rolling_training_early_stopping(ticker, window_size=200, step_size=10, n_steps=5)


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration
########################
ticker = 'COST'
start_date = '2020-01-01'
end_date = '2024-01-01'

# Hyperparameters
n_steps = 10
window_size = 200
step_size = 10
initial_epochs = 100
update_epochs = 100
initial_lr = 0.0002
update_lr = 0.0002
train_ratio = 0.9
n_units = 32
batch_size = 8

# If you change features or other logic, do so below
if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")


def load_full_data_helper(ticker, start_date, end_date, window_size):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    # Next day prediction setup
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    # Simple features
    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates


def load_full_data(ticker, start_date, end_date, window_size):
    X_feat, y_raw, dates = load_full_data_helper(ticker, start_date, end_date, window_size)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates


def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)

    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)


def create_lstm_model(n_units, learning_rate, n_steps, n_features, n_layers=1):
    inputs = Input(shape=(n_steps, n_features))
    x = LSTM(n_units, activation='relu', return_sequences=(n_layers > 1))(inputs)
    if n_layers > 1:
        x = LSTM(n_units, activation='relu')(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    # We'll compile after loading weights if needed
    return model


def train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                    initial_training=False, prev_weights=None):
    total_samples = len(X_samples)
    global train_ratio
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if X_train.size == 0 or X_val.size == 0:
        return None, np.nan

    n_steps_local = X_train.shape[1]
    n_features = X_train.shape[2]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    model = create_lstm_model(n_units, lr, n_steps_local, n_features, n_layers=1)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape


def rolling_training_early_stopping(ticker, window_size, step_size, n_steps):
    X_full, y_full, dates_full = load_full_data(ticker, start_date, end_date, window_size)
    if X_full is None:
        print("[WARN] Not enough data.")
        return

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print("[WARN] Initial window not valid.")
        return
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    print("[INFO] Training initial window with Early Stopping...")
    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=True, prev_weights=None)
    print(f"[RESULT] Initial window validation MAPE: {val_mape:.2f}%")

    weights_path = "rolling_error_analysis_results/cost_initial_es.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print("[INFO] No more windows can be extracted.")
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: training on new window with Early Stopping...")
        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=weights_path)
        print(f"[RESULT] Window validation MAPE: {val_mape:.2f}%")

        weights_path = f"rolling_error_analysis_results/cost_iteration_{iteration}_es.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] Predicted last sample of window: Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv("rolling_error_analysis_results/cost_rolling_results_es.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training with Early Stopping completed for {ticker}. Overall MAPE: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] Total elapsed time: {elapsed:.2f}s")

    worst_10 = df_results.sort_values('MAPE', ascending=False).head(10)
    print("[INFO] Top 10 worst predictions in rolling scenario (Early Stopping):")
    print(worst_10[['Date', 'Actual', 'Predicted', 'MAPE']])

if __name__ == "__main__":
    # Just run with the current top-level defined parameters:
    rolling_training_early_stopping(ticker, window_size, step_size, n_steps)


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration
########################
ticker = 'COST'
start_date = '2020-01-01'
end_date = '2024-01-01'

# Hyperparameters
n_steps = 5
window_size = 200
step_size = 10
initial_epochs = 100
update_epochs = 100
initial_lr = 0.0005
update_lr = 0.0002
train_ratio = 0.9
n_units = 16
batch_size = 4

# If you change features or other logic, do so below
if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")


def load_full_data_helper(ticker, start_date, end_date, window_size):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    # Next day prediction setup
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    # Simple features
    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates


def load_full_data(ticker, start_date, end_date, window_size):
    X_feat, y_raw, dates = load_full_data_helper(ticker, start_date, end_date, window_size)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates


def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)

    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)


def create_lstm_model(n_units, learning_rate, n_steps, n_features, n_layers=1):
    inputs = Input(shape=(n_steps, n_features))
    x = LSTM(n_units, activation='relu', return_sequences=(n_layers > 1))(inputs)
    if n_layers > 1:
        x = LSTM(n_units, activation='relu')(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    # We'll compile after loading weights if needed
    return model


def train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                    initial_training=False, prev_weights=None):
    total_samples = len(X_samples)
    global train_ratio
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if X_train.size == 0 or X_val.size == 0:
        return None, np.nan

    n_steps_local = X_train.shape[1]
    n_features = X_train.shape[2]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    model = create_lstm_model(n_units, lr, n_steps_local, n_features, n_layers=1)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape


def rolling_training_early_stopping(ticker, window_size, step_size, n_steps):
    X_full, y_full, dates_full = load_full_data(ticker, start_date, end_date, window_size)
    if X_full is None:
        print("[WARN] Not enough data.")
        return

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print("[WARN] Initial window not valid.")
        return
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    print("[INFO] Training initial window with Early Stopping...")
    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=True, prev_weights=None)
    print(f"[RESULT] Initial window validation MAPE: {val_mape:.2f}%")

    weights_path = "rolling_error_analysis_results/cost_initial_es.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print("[INFO] No more windows can be extracted.")
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: training on new window with Early Stopping...")
        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=weights_path)
        print(f"[RESULT] Window validation MAPE: {val_mape:.2f}%")

        weights_path = f"rolling_error_analysis_results/cost_iteration_{iteration}_es.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] Predicted last sample of window: Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv("rolling_error_analysis_results/cost_rolling_results_es.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training with Early Stopping completed for {ticker}. Overall MAPE: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] Total elapsed time: {elapsed:.2f}s")

    worst_10 = df_results.sort_values('MAPE', ascending=False).head(10)
    print("[INFO] Top 10 worst predictions in rolling scenario (Early Stopping):")
    print(worst_10[['Date', 'Actual', 'Predicted', 'MAPE']])

if __name__ == "__main__":
    # Just run with the current top-level defined parameters:
    rolling_training_early_stopping(ticker, window_size, step_size, n_steps)


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration
########################

stocks = [
    'COST', 'UL', 'AMGN', 'UNH', 'WAT', 'UPS', 'LPX', 'WM', 'NVDA',
    'GOOGL', 'MSFT', 'AXP', 'BLK', 'BRK-B', 'NEE', 'XOM', 'CNI'
]

start_date = '2020-01-01'
end_date = '2024-01-01'

# Hyperparameters
n_steps = 5
window_size = 200
step_size = 10
initial_epochs = 100
update_epochs = 100
initial_lr = 0.0005
update_lr = 0.0002
train_ratio = 0.9
n_units = 16
batch_size = 4

if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")

def load_full_data_helper(ticker, start_date, end_date, window_size):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    # Next day prediction setup
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates

def load_full_data(ticker, start_date, end_date, window_size):
    X_feat, y_raw, dates = load_full_data_helper(ticker, start_date, end_date, window_size)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates

def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)

    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)

def create_lstm_model(n_units, learning_rate, n_steps, n_features, n_layers=1):
    inputs = Input(shape=(n_steps, n_features))
    x = LSTM(n_units, activation='relu', return_sequences=(n_layers > 1))(inputs)
    if n_layers > 1:
        x = LSTM(n_units, activation='relu')(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    return model

def train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                    initial_training=False, prev_weights=None):
    total_samples = len(X_samples)
    global train_ratio
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if X_train.size == 0 or X_val.size == 0:
        return None, np.nan

    n_steps_local = X_train.shape[1]
    n_features = X_train.shape[2]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    model = create_lstm_model(n_units, lr, n_steps_local, n_features, n_layers=1)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape

def rolling_training_early_stopping(ticker, window_size, step_size, n_steps):
    X_full, y_full, dates_full = load_full_data(ticker, start_date, end_date, window_size)
    if X_full is None:
        print(f"[WARN] Not enough data for {ticker}.")
        return None

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print(f"[WARN] Initial window not valid for {ticker}.")
        return None
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    print(f"[INFO] Training initial window with Early Stopping for {ticker}...")
    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=True, prev_weights=None)
    print(f"[RESULT] {ticker}: Initial window validation MAPE: {val_mape:.2f}%")

    weights_path = f"rolling_error_analysis_results/{ticker}_initial_es.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print(f"[INFO] No more windows can be extracted for {ticker}.")
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: {ticker}, training on new window with Early Stopping...")
        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=weights_path)
        print(f"[RESULT] {ticker}: Window validation MAPE: {val_mape:.2f}%")

        weights_path = f"rolling_error_analysis_results/{ticker}_iteration_{iteration}_es.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] {ticker}: Predicted last sample of window: Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv(f"rolling_error_analysis_results/{ticker}_rolling_results_es.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training with Early Stopping completed for {ticker}. Overall MAPE: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] {ticker}: Total elapsed time: {elapsed:.2f}s")

    # Return the overall MAPE so we can store it in a CSV by stock
    return overall_mape

if __name__ == "__main__":
    # Run the code for each stock and save their MAPEs to a CSV
    stock_list = [
        'COST', 'UL', 'AMGN', 'UNH', 'WAT', 'UPS', 'LPX', 'WM', 'NVDA',
        'GOOGL', 'MSFT', 'AXP', 'BLK', 'BRK-B', 'NEE', 'XOM', 'CNI'
    ]

    results = []
    for s in stock_list:
        mape = rolling_training_early_stopping(s, window_size, step_size, n_steps)
        if mape is not None:
            results.append({'Stock': s, 'Overall_MAPE': mape})
        else:
            results.append({'Stock': s, 'Overall_MAPE': np.nan})

    results_df = pd.DataFrame(results)
    results_df.to_csv('stock_mapes.csv', index=False)
    print("[INFO] All stock MAPEs saved to stock_mapes.csv")


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.metrics import mean_absolute_percentage_error
import os
import math
import time

stocks = [
    'COST', 'UL', 'AMGN', 'UNH', 'WAT', 'UPS', 'LPX', 'WM', 'NVDA',
    'GOOGL', 'MSFT', 'AXP', 'BLK', 'BRK-B', 'NEE', 'XOM', 'CNI'
]

start_date = '2020-01-01'
end_date = '2024-01-01'

n_steps = 5
window_size = 200
step_size = 10

if not os.path.exists("baseline_sma_results"):
    os.makedirs("baseline_sma_results")

def load_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)
    X = stock_data['Close'].values
    y = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values
    return X, y, dates

def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None
    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]
    if X_samples.size == 0:
        return None, None, None, None
    return X_samples, y_samples, sample_dates, (X_window, y_window, dates_window)

def baseline_sma_prediction(X_samples, n_steps):
    predictions = []
    for i in range(len(X_samples)):
        pred = np.mean(X_samples[i])
        predictions.append(pred)
    return np.array(predictions)

def rolling_baseline_sma(ticker, window_size, step_size, n_steps):
    X_full, y_full, dates_full = load_data(ticker, start_date, end_date)
    if X_full is None:
        print(f"[WARN] Not enough data for {ticker}.")
        return None

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print(f"[WARN] Initial window not valid for {ticker}.")
        return None
    X_samples, y_samples, sample_dates, _ = window_data
    predictions = baseline_sma_prediction(X_samples, n_steps)

    all_predictions = list(predictions)
    all_actuals = list(y_samples)
    all_dates = list(sample_dates)

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print(f"[INFO] No more windows for {ticker}.")
            break
        X_samples, y_samples, sample_dates, _ = window_data
        predictions = baseline_sma_prediction(X_samples, n_steps)
        all_predictions.extend(predictions)
        all_actuals.extend(y_samples)
        all_dates.extend(sample_dates)
        iteration += 1

    all_predictions = np.array(all_predictions)
    all_actuals = np.array(all_actuals)
    mape = mean_absolute_percentage_error(all_actuals, all_predictions)*100.0

    df_results = pd.DataFrame({
        'Date': all_dates,
        'Actual': all_actuals,
        'Predicted': all_predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0
    df_results.to_csv(f"baseline_sma_results/{ticker}_rolling_baseline_sma_results.csv", index=False)

    print(f"[INFO] Baseline SMA {ticker}: Overall MAPE: {mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] {ticker}: Elapsed time: {elapsed:.2f}s")
    return mape

if __name__ == "__main__":
    results = []
    for s in stocks:
        mape = rolling_baseline_sma(s, window_size, step_size, n_steps)
        if mape is not None:
            results.append({'Stock': s, 'Overall_MAPE': mape})
        else:
            results.append({'Stock': s, 'Overall_MAPE': np.nan})

    results_df = pd.DataFrame(results)
    results_df.to_csv('stock_mapes_sma_baseline.csv', index=False)
    print("[INFO] All SMA baseline MAPEs saved to stock_mapes_sma_baseline.csv")


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration
########################
ticker = 'NVDA'
start_date = '2020-01-01'
end_date = '2024-01-01'
1
# Hyperparameters
n_steps = 5
window_size = 200
step_size = 10
initial_epochs = 100
update_epochs = 100
initial_lr = 0.0005
update_lr = 0.0002
train_ratio = 0.9
n_units = 16
batch_size = 4

# If you change features or other logic, do so below
if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")


def load_full_data_helper(ticker, start_date, end_date, window_size):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    # Next day prediction setup
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    # Simple features
    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates


def load_full_data(ticker, start_date, end_date, window_size):
    X_feat, y_raw, dates = load_full_data_helper(ticker, start_date, end_date, window_size)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates


def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)

    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)


def create_lstm_model(n_units, learning_rate, n_steps, n_features, n_layers=1):
    inputs = Input(shape=(n_steps, n_features))
    x = LSTM(n_units, activation='relu', return_sequences=(n_layers > 1))(inputs)
    if n_layers > 1:
        x = LSTM(n_units, activation='relu')(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    # We'll compile after loading weights if needed
    return model


def train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                    initial_training=False, prev_weights=None):
    total_samples = len(X_samples)
    global train_ratio
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if X_train.size == 0 or X_val.size == 0:
        return None, np.nan

    n_steps_local = X_train.shape[1]
    n_features = X_train.shape[2]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    model = create_lstm_model(n_units, lr, n_steps_local, n_features, n_layers=1)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape


def rolling_training_early_stopping(ticker, window_size, step_size, n_steps):
    X_full, y_full, dates_full = load_full_data(ticker, start_date, end_date, window_size)
    if X_full is None:
        print("[WARN] Not enough data.")
        return

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print("[WARN] Initial window not valid.")
        return
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    print("[INFO] Training initial window with Early Stopping...")
    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=True, prev_weights=None)
    print(f"[RESULT] Initial window validation MAPE: {val_mape:.2f}%")

    weights_path = "rolling_error_analysis_results/cost_initial_es.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print("[INFO] No more windows can be extracted.")
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: training on new window with Early Stopping...")
        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=weights_path)
        print(f"[RESULT] Window validation MAPE: {val_mape:.2f}%")

        weights_path = f"rolling_error_analysis_results/cost_iteration_{iteration}_es.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] Predicted last sample of window: Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv("rolling_error_analysis_results/cost_rolling_results_es.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training with Early Stopping completed for {ticker}. Overall MAPE: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] Total elapsed time: {elapsed:.2f}s")

    worst_10 = df_results.sort_values('MAPE', ascending=False).head(10)
    print("[INFO] Top 10 worst predictions in rolling scenario (Early Stopping):")
    print(worst_10[['Date', 'Actual', 'Predicted', 'MAPE']])

if __name__ == "__main__":
    # Just run with the current top-level defined parameters:
    rolling_training_early_stopping(ticker, window_size, step_size, n_steps)


In [12]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration
########################
ticker = 'NVDA'
start_date = '2020-01-01'
end_date = '2024-01-01'
1
# Hyperparameters
n_steps = 2
window_size = 5
step_size = 2
initial_epochs = 100
update_epochs = 100
initial_lr = 0.0005
update_lr = 0.00005
train_ratio = 0.9
n_units = 16
batch_size = 4

# If you change features or other logic, do so below
if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")


def load_full_data_helper(ticker, start_date, end_date, window_size):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    # Next day prediction setup
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    # Simple features
    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates


def load_full_data(ticker, start_date, end_date, window_size):
    X_feat, y_raw, dates = load_full_data_helper(ticker, start_date, end_date, window_size)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates


def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)

    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)


def create_lstm_model(n_units, learning_rate, n_steps, n_features, n_layers=1):
    inputs = Input(shape=(n_steps, n_features))
    x = LSTM(n_units, activation='relu', return_sequences=(n_layers > 1))(inputs)
    if n_layers > 1:
        x = LSTM(n_units, activation='relu')(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    # We'll compile after loading weights if needed
    return model


def train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                    initial_training=False, prev_weights=None):
    total_samples = len(X_samples)
    global train_ratio
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if X_train.size == 0 or X_val.size == 0:
        return None, np.nan

    n_steps_local = X_train.shape[1]
    n_features = X_train.shape[2]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    model = create_lstm_model(n_units, lr, n_steps_local, n_features, n_layers=1)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape


def rolling_training_early_stopping(ticker, window_size, step_size, n_steps):
    X_full, y_full, dates_full = load_full_data(ticker, start_date, end_date, window_size)
    if X_full is None:
        print("[WARN] Not enough data.")
        return

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print("[WARN] Initial window not valid.")
        return
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    print("[INFO] Training initial window with Early Stopping...")
    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=True, prev_weights=None)
    print(f"[RESULT] Initial window validation MAPE: {val_mape:.2f}%")

    weights_path = "rolling_error_analysis_results/cost_initial_es.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print("[INFO] No more windows can be extracted.")
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: training on new window with Early Stopping...")
        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=weights_path)
        print(f"[RESULT] Window validation MAPE: {val_mape:.2f}%")

        weights_path = f"rolling_error_analysis_results/cost_iteration_{iteration}_es.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] Predicted last sample of window: Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv("rolling_error_analysis_results/cost_rolling_results_es.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training with Early Stopping completed for {ticker}. Overall MAPE: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] Total elapsed time: {elapsed:.2f}s")

    worst_10 = df_results.sort_values('MAPE', ascending=False).head(10)
    print("[INFO] Top 10 worst predictions in rolling scenario (Early Stopping):")
    print(worst_10[['Date', 'Actual', 'Predicted', 'MAPE']])

if __name__ == "__main__":
    # Just run with the current top-level defined parameters:
    rolling_training_early_stopping(ticker, window_size, step_size, n_steps)


[*********************100%%**********************]  1 of 1 completed
[INFO] Training initial window with Early Stopping...
[RESULT] Initial window validation MAPE: 1.28%

[INFO] Iteration 1: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 3.60%
[INFO] Predicted last sample of window: Actual=6.30, Pred=6.07, MAPE=3.60%

[INFO] Iteration 2: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.01%
[INFO] Predicted last sample of window: Actual=6.14, Pred=6.14, MAPE=0.01%

[INFO] Iteration 3: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.16%
[INFO] Predicted last sample of window: Actual=6.23, Pred=6.22, MAPE=0.16%

[INFO] Iteration 4: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.44%
[INFO] Predicted last sample of window: Actual=6.25, Pred=6.23, MAPE=0.44%

[INFO] Iteration 5: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.14%


[RESULT] Window validation MAPE: 2.38%
[INFO] Predicted last sample of window: Actual=8.81, Pred=9.02, MAPE=2.38%

[INFO] Iteration 46: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.04%
[INFO] Predicted last sample of window: Actual=8.78, Pred=8.78, MAPE=0.04%

[INFO] Iteration 47: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.03%
[INFO] Predicted last sample of window: Actual=8.72, Pred=8.81, MAPE=1.03%

[INFO] Iteration 48: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.79%
[INFO] Predicted last sample of window: Actual=8.49, Pred=8.25, MAPE=2.79%

[INFO] Iteration 49: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.76%
[INFO] Predicted last sample of window: Actual=8.81, Pred=8.74, MAPE=0.76%

[INFO] Iteration 50: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.50%
[INFO] Predicted last sample of window: Actual=8.77

[RESULT] Window validation MAPE: 2.48%
[INFO] Predicted last sample of window: Actual=12.87, Pred=12.55, MAPE=2.48%

[INFO] Iteration 91: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.12%
[INFO] Predicted last sample of window: Actual=13.23, Pred=13.37, MAPE=1.12%

[INFO] Iteration 92: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.51%
[INFO] Predicted last sample of window: Actual=13.61, Pred=13.27, MAPE=2.51%

[INFO] Iteration 93: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.18%
[INFO] Predicted last sample of window: Actual=13.64, Pred=13.62, MAPE=0.18%

[INFO] Iteration 94: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.17%
[INFO] Predicted last sample of window: Actual=13.96, Pred=13.66, MAPE=2.17%

[INFO] Iteration 95: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.07%
[INFO] Predicted last sample of window: A

[RESULT] Window validation MAPE: 0.73%
[INFO] Predicted last sample of window: Actual=13.66, Pred=13.56, MAPE=0.73%

[INFO] Iteration 136: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 5.79%
[INFO] Predicted last sample of window: Actual=14.44, Pred=13.60, MAPE=5.79%

[INFO] Iteration 137: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.55%
[INFO] Predicted last sample of window: Actual=14.76, Pred=14.99, MAPE=1.55%

[INFO] Iteration 138: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.97%
[INFO] Predicted last sample of window: Actual=14.96, Pred=14.82, MAPE=0.97%

[INFO] Iteration 139: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.50%
[INFO] Predicted last sample of window: Actual=14.91, Pred=14.98, MAPE=0.50%

[INFO] Iteration 140: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.13%
[INFO] Predicted last sample of wind

[RESULT] Window validation MAPE: 2.20%
[INFO] Predicted last sample of window: Actual=17.79, Pred=18.18, MAPE=2.20%

[INFO] Iteration 181: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 4.39%
[INFO] Predicted last sample of window: Actual=18.66, Pred=17.84, MAPE=4.39%

[INFO] Iteration 182: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 8.25%
[INFO] Predicted last sample of window: Actual=18.43, Pred=19.95, MAPE=8.25%

[INFO] Iteration 183: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.94%
[INFO] Predicted last sample of window: Actual=19.06, Pred=18.69, MAPE=1.94%

[INFO] Iteration 184: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.12%
[INFO] Predicted last sample of window: Actual=19.03, Pred=19.05, MAPE=0.12%

[INFO] Iteration 185: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 3.40%
[INFO] Predicted last sample of wind

...

[RESULT] Window validation MAPE: 0.46%
[INFO] Predicted last sample of window: Actual=15.08, Pred=15.15, MAPE=0.46%

[INFO] Iteration 316: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.32%
[INFO] Predicted last sample of window: Actual=15.37, Pred=15.17, MAPE=1.32%

[INFO] Iteration 317: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 4.40%
[INFO] Predicted last sample of window: Actual=16.10, Pred=15.39, MAPE=4.40%

[INFO] Iteration 318: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 9.54%
[INFO] Predicted last sample of window: Actual=17.81, Pred=16.11, MAPE=9.54%

[INFO] Iteration 319: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.56%
[INFO] Predicted last sample of window: Actual=17.32, Pred=17.42, MAPE=0.56%

[INFO] Iteration 320: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 4.41%
[INFO] Predicted last sample of wind

[RESULT] Window validation MAPE: 2.59%
[INFO] Predicted last sample of window: Actual=15.91, Pred=16.32, MAPE=2.59%

[INFO] Iteration 361: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.53%
[INFO] Predicted last sample of window: Actual=15.41, Pred=15.80, MAPE=2.53%

[INFO] Iteration 362: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 3.41%
[INFO] Predicted last sample of window: Actual=16.04, Pred=15.49, MAPE=3.41%

[INFO] Iteration 363: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.19%
[INFO] Predicted last sample of window: Actual=16.27, Pred=16.24, MAPE=0.19%

[INFO] Iteration 364: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.57%
[INFO] Predicted last sample of window: Actual=15.64, Pred=16.04, MAPE=2.57%

[INFO] Iteration 365: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 5.16%
[INFO] Predicted last sample of wind

[RESULT] Window validation MAPE: 1.76%
[INFO] Predicted last sample of window: Actual=26.98, Pred=26.51, MAPE=1.76%

[INFO] Iteration 406: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.82%
[INFO] Predicted last sample of window: Actual=27.78, Pred=27.27, MAPE=1.82%

[INFO] Iteration 407: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.05%
[INFO] Predicted last sample of window: Actual=27.45, Pred=27.47, MAPE=0.05%

[INFO] Iteration 408: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.17%
[INFO] Predicted last sample of window: Actual=27.04, Pred=27.08, MAPE=0.17%

[INFO] Iteration 409: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.00%
[INFO] Predicted last sample of window: Actual=27.17, Pred=27.17, MAPE=0.00%

[INFO] Iteration 410: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.44%
[INFO] Predicted last sample of wind

[RESULT] Window validation MAPE: 0.00%
[INFO] Predicted last sample of window: Actual=44.66, Pred=44.67, MAPE=0.00%

[INFO] Iteration 451: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.05%
[INFO] Predicted last sample of window: Actual=42.39, Pred=43.26, MAPE=2.05%

[INFO] Iteration 452: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 3.11%
[INFO] Predicted last sample of window: Actual=43.75, Pred=42.39, MAPE=3.11%

[INFO] Iteration 453: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.25%
[INFO] Predicted last sample of window: Actual=43.49, Pred=43.59, MAPE=0.25%

[INFO] Iteration 454: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.42%
[INFO] Predicted last sample of window: Actual=43.30, Pred=43.48, MAPE=0.42%

[INFO] Iteration 455: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 3.30%
[INFO] Predicted last sample of wind

[RESULT] Window validation MAPE: 1.11%
[INFO] Predicted last sample of window: Actual=48.35, Pred=47.81, MAPE=1.11%

[INFO] Iteration 496: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 3.27%
[INFO] Predicted last sample of window: Actual=50.08, Pred=48.44, MAPE=3.27%

[INFO] Iteration 497: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.02%
[INFO] Predicted last sample of window: Actual=48.11, Pred=49.08, MAPE=2.02%

[INFO] Iteration 498: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.36%
[INFO] Predicted last sample of window: Actual=48.83, Pred=49.01, MAPE=0.36%

[INFO] Iteration 499: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.74%
[INFO] Predicted last sample of window: Actual=49.42, Pred=49.05, MAPE=0.74%

[INFO] Iteration 500: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.14%
[INFO] Predicted last sample of wind

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration
########################

stocks = [
    'COST', 'UL', 'AMGN', 'UNH', 'WAT', 'UPS', 'LPX', 'WM', 'NVDA',
    'GOOGL', 'MSFT', 'AXP', 'BLK', 'BRK-B', 'NEE', 'XOM', 'CNI'
]

start_date = '2020-01-01'
end_date = '2024-01-01'

# Hyperparameters
n_steps = 2
window_size = 5
step_size = 2
initial_epochs = 100
update_epochs = 100
initial_lr = 0.0005
update_lr = 0.00005
train_ratio = 0.9
n_units = 16
batch_size = 4

if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")

def load_full_data_helper(ticker, start_date, end_date, window_size):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    # Next day prediction setup
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates

def load_full_data(ticker, start_date, end_date, window_size):
    X_feat, y_raw, dates = load_full_data_helper(ticker, start_date, end_date, window_size)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates

def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)

    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)

def create_lstm_model(n_units, learning_rate, n_steps, n_features, n_layers=1):
    inputs = Input(shape=(n_steps, n_features))
    x = LSTM(n_units, activation='relu', return_sequences=(n_layers > 1))(inputs)
    if n_layers > 1:
        x = LSTM(n_units, activation='relu')(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    return model

def train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                    initial_training=False, prev_weights=None):
    total_samples = len(X_samples)
    global train_ratio
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if X_train.size == 0 or X_val.size == 0:
        return None, np.nan

    n_steps_local = X_train.shape[1]
    n_features = X_train.shape[2]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    model = create_lstm_model(n_units, lr, n_steps_local, n_features, n_layers=1)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape

def rolling_training_early_stopping(ticker, window_size, step_size, n_steps):
    X_full, y_full, dates_full = load_full_data(ticker, start_date, end_date, window_size)
    if X_full is None:
        print(f"[WARN] Not enough data for {ticker}.")
        return None

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print(f"[WARN] Initial window not valid for {ticker}.")
        return None
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    print(f"[INFO] Training initial window with Early Stopping for {ticker}...")
    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=True, prev_weights=None)
    print(f"[RESULT] {ticker}: Initial window validation MAPE: {val_mape:.2f}%")

    weights_path = f"rolling_error_analysis_results/{ticker}_initial_es.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print(f"[INFO] No more windows can be extracted for {ticker}.")
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: {ticker}, training on new window with Early Stopping...")
        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=weights_path)
        print(f"[RESULT] {ticker}: Window validation MAPE: {val_mape:.2f}%")

        weights_path = f"rolling_error_analysis_results/{ticker}_iteration_{iteration}_es.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] {ticker}: Predicted last sample of window: Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv(f"rolling_error_analysis_results/{ticker}_rolling_results_es.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training with Early Stopping completed for {ticker}. Overall MAPE: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] {ticker}: Total elapsed time: {elapsed:.2f}s")

    # Return the overall MAPE so we can store it in a CSV by stock
    return overall_mape

if __name__ == "__main__":
    # Run the code for each stock and save their MAPEs to a CSV
    stock_list = [
        'COST', 'UL', 'AMGN', 'UNH', 'WAT', 'UPS', 'LPX', 'WM', 'NVDA',
        'GOOGL', 'MSFT', 'AXP', 'BLK', 'BRK-B', 'NEE', 'XOM', 'CNI'
    ]

    results = []
    for s in stock_list:
        mape = rolling_training_early_stopping(s, window_size, step_size, n_steps)
        if mape is not None:
            results.append({'Stock': s, 'Overall_MAPE': mape})
        else:
            results.append({'Stock': s, 'Overall_MAPE': np.nan})

    results_df = pd.DataFrame(results)
    results_df.to_csv('stock_mapes.csv', index=False)
    print("[INFO] All stock MAPEs saved to stock_mapes.csv")


[*********************100%%**********************]  1 of 1 completed
[INFO] Training initial window with Early Stopping for COST...
[RESULT] COST: Initial window validation MAPE: 2.51%

[INFO] Iteration 1: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.98%
[INFO] COST: Predicted last sample of window: Actual=299.87, Pred=296.94, MAPE=0.98%

[INFO] Iteration 2: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.36%
[INFO] COST: Predicted last sample of window: Actual=300.82, Pred=299.75, MAPE=0.36%

[INFO] Iteration 3: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 1.66%
[INFO] COST: Predicted last sample of window: Actual=304.68, Pred=299.63, MAPE=1.66%

[INFO] Iteration 4: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 2.50%
[INFO] COST: Predicted last sample of window: Actual=311.89, Pred=304.08, MAPE=2.50%

[I

[RESULT] COST: Window validation MAPE: 1.13%
[INFO] COST: Predicted last sample of window: Actual=307.92, Pred=304.46, MAPE=1.13%

[INFO] Iteration 41: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.00%
[INFO] COST: Predicted last sample of window: Actual=305.00, Pred=304.99, MAPE=0.00%

[INFO] Iteration 42: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.73%
[INFO] COST: Predicted last sample of window: Actual=310.33, Pred=308.08, MAPE=0.73%

[INFO] Iteration 43: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.95%
[INFO] COST: Predicted last sample of window: Actual=302.14, Pred=305.02, MAPE=0.95%

[INFO] Iteration 44: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.67%
[INFO] COST: Predicted last sample of window: Actual=299.21, Pred=301.22, MAPE=0.67%

[INFO] Iteration 45: COST, training on new window wit

[RESULT] COST: Window validation MAPE: 0.15%
[INFO] COST: Predicted last sample of window: Actual=347.66, Pred=347.15, MAPE=0.15%

[INFO] Iteration 82: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 2.84%
[INFO] COST: Predicted last sample of window: Actual=358.86, Pred=348.67, MAPE=2.84%

[INFO] Iteration 83: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 5.33%
[INFO] COST: Predicted last sample of window: Actual=346.57, Pred=328.10, MAPE=5.33%

[INFO] Iteration 84: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.00%
[INFO] COST: Predicted last sample of window: Actual=346.43, Pred=346.44, MAPE=0.00%

[INFO] Iteration 85: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.14%
[INFO] COST: Predicted last sample of window: Actual=339.13, Pred=339.61, MAPE=0.14%

[INFO] Iteration 86: COST, training on new window wit

[RESULT] COST: Window validation MAPE: 1.88%
[INFO] COST: Predicted last sample of window: Actual=371.06, Pred=364.10, MAPE=1.88%

[INFO] Iteration 123: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.15%
[INFO] COST: Predicted last sample of window: Actual=374.45, Pred=373.89, MAPE=0.15%

[INFO] Iteration 124: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 1.41%
[INFO] COST: Predicted last sample of window: Actual=380.15, Pred=374.77, MAPE=1.41%

[INFO] Iteration 125: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 1.65%
[INFO] COST: Predicted last sample of window: Actual=370.02, Pred=376.13, MAPE=1.65%

[INFO] Iteration 126: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.33%
[INFO] COST: Predicted last sample of window: Actual=369.94, Pred=368.74, MAPE=0.33%

[INFO] Iteration 127: COST, training on new windo

[RESULT] COST: Window validation MAPE: 0.18%
[INFO] COST: Predicted last sample of window: Actual=373.28, Pred=372.61, MAPE=0.18%

[INFO] Iteration 163: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.33%
[INFO] COST: Predicted last sample of window: Actual=370.21, Pred=371.42, MAPE=0.33%

[INFO] Iteration 164: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 1.12%
[INFO] COST: Predicted last sample of window: Actual=373.54, Pred=369.34, MAPE=1.12%

[INFO] Iteration 165: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.37%
[INFO] COST: Predicted last sample of window: Actual=379.32, Pred=377.93, MAPE=0.37%

[INFO] Iteration 166: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.40%
[INFO] COST: Predicted last sample of window: Actual=372.50, Pred=373.98, MAPE=0.40%

[INFO] Iteration 167: COST, training on new windo

...

[RESULT] COST: Window validation MAPE: 0.65%
[INFO] COST: Predicted last sample of window: Actual=490.85, Pred=487.68, MAPE=0.65%

[INFO] Iteration 403: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.02%
[INFO] COST: Predicted last sample of window: Actual=487.76, Pred=487.83, MAPE=0.02%

[INFO] Iteration 404: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.20%
[INFO] COST: Predicted last sample of window: Actual=493.22, Pred=492.24, MAPE=0.20%

[INFO] Iteration 405: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.09%
[INFO] COST: Predicted last sample of window: Actual=490.87, Pred=490.45, MAPE=0.09%

[INFO] Iteration 406: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.91%
[INFO] COST: Predicted last sample of window: Actual=496.87, Pred=492.34, MAPE=0.91%

[INFO] Iteration 407: COST, training on new windo

[RESULT] COST: Window validation MAPE: 2.08%
[INFO] COST: Predicted last sample of window: Actual=552.96, Pred=541.46, MAPE=2.08%

[INFO] Iteration 443: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.03%
[INFO] COST: Predicted last sample of window: Actual=554.11, Pred=554.26, MAPE=0.03%

[INFO] Iteration 444: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.44%
[INFO] COST: Predicted last sample of window: Actual=557.86, Pred=555.42, MAPE=0.44%

[INFO] Iteration 445: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.84%
[INFO] COST: Predicted last sample of window: Actual=565.19, Pred=560.43, MAPE=0.84%

[INFO] Iteration 446: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.61%
[INFO] COST: Predicted last sample of window: Actual=561.83, Pred=565.28, MAPE=0.61%

[INFO] Iteration 447: COST, training on new windo

[RESULT] COST: Window validation MAPE: 0.90%
[INFO] COST: Predicted last sample of window: Actual=571.27, Pred=566.14, MAPE=0.90%

[INFO] Iteration 483: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.92%
[INFO] COST: Predicted last sample of window: Actual=563.27, Pred=568.48, MAPE=0.92%

[INFO] Iteration 484: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.49%
[INFO] COST: Predicted last sample of window: Actual=578.23, Pred=575.42, MAPE=0.49%

[INFO] Iteration 485: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 1.90%
[INFO] COST: Predicted last sample of window: Actual=596.78, Pred=585.44, MAPE=1.90%

[INFO] Iteration 486: COST, training on new window with Early Stopping...
[RESULT] COST: Window validation MAPE: 0.41%
[INFO] COST: Predicted last sample of window: Actual=577.15, Pred=579.50, MAPE=0.41%

[INFO] Iteration 487: COST, training on new windo

[RESULT] UL: Window validation MAPE: 2.70%
[INFO] UL: Predicted last sample of window: Actual=49.97, Pred=51.32, MAPE=2.70%

[INFO] Iteration 23: UL, training on new window with Early Stopping...
[RESULT] UL: Window validation MAPE: 0.90%
[INFO] UL: Predicted last sample of window: Actual=50.38, Pred=49.93, MAPE=0.90%

[INFO] Iteration 24: UL, training on new window with Early Stopping...
[RESULT] UL: Window validation MAPE: 1.26%
[INFO] UL: Predicted last sample of window: Actual=50.66, Pred=50.02, MAPE=1.26%

[INFO] Iteration 25: UL, training on new window with Early Stopping...
[RESULT] UL: Window validation MAPE: 10.04%
[INFO] UL: Predicted last sample of window: Actual=44.62, Pred=49.10, MAPE=10.04%

[INFO] Iteration 26: UL, training on new window with Early Stopping...
[RESULT] UL: Window validation MAPE: 0.27%
[INFO] UL: Predicted last sample of window: Actual=47.61, Pred=47.48, MAPE=0.27%

[INFO] Iteration 27: UL, training on new window with Early Stopping...
[RESULT] UL: Windo

[RESULT] UL: Window validation MAPE: 0.68%
[INFO] UL: Predicted last sample of window: Actual=55.00, Pred=54.63, MAPE=0.68%

[INFO] Iteration 65: UL, training on new window with Early Stopping...
[RESULT] UL: Window validation MAPE: 0.16%
[INFO] UL: Predicted last sample of window: Actual=55.16, Pred=55.07, MAPE=0.16%

[INFO] Iteration 66: UL, training on new window with Early Stopping...
[RESULT] UL: Window validation MAPE: 0.07%
[INFO] UL: Predicted last sample of window: Actual=55.27, Pred=55.23, MAPE=0.07%

[INFO] Iteration 67: UL, training on new window with Early Stopping...
[RESULT] UL: Window validation MAPE: 0.39%
[INFO] UL: Predicted last sample of window: Actual=55.85, Pred=55.63, MAPE=0.39%

[INFO] Iteration 68: UL, training on new window with Early Stopping...
[RESULT] UL: Window validation MAPE: 0.00%
[INFO] UL: Predicted last sample of window: Actual=60.15, Pred=60.15, MAPE=0.00%

[INFO] Iteration 69: UL, training on new window with Early Stopping...
[RESULT] UL: Window 

In [4]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
import os
import time

stocks = [
    'COST', 'UL', 'AMGN', 'UNH', 'WAT', 'UPS', 'LPX', 'WM', 'NVDA',
    'GOOGL', 'MSFT', 'AXP', 'BLK', 'BRK-B', 'NEE', 'XOM', 'CNI'
]

start_date = '2020-01-01'
end_date = '2024-01-01'

window_size = 5
step_size = 2

if not os.path.exists("baseline_results"):
    os.makedirs("baseline_results")

def load_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)
    X = stock_data['Close'].values
    y = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values
    return X, y, dates

def extract_window(X_full, y_full, dates_full, start_idx, window_size):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None
    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]
    return X_window, y_window, dates_window

def sma_prediction(X_window):
    return np.mean(X_window[-2:])

def ema_prediction(X_window):
    ema = 0
    alpha = 2 / (len(X_window[-2:]) + 1)
    for i, price in enumerate(X_window[-2:]):
        ema += price * (alpha * (1 - alpha) ** (len(X_window[-2:]) - i - 1))
    return ema

def linear_regression_prediction(X_window):
    if len(X_window) < 2:
        return np.nan
    X = np.array([-2, -1]).reshape(-1, 1)  # Days n-2 and n-1
    y = X_window[-2:]  # Prices for days n-2 and n-1
    model = LinearRegression()
    model.fit(X, y)
    return model.predict([[0]])[0]  # Predict day n

def rolling_predictions(ticker, window_size, step_size):
    X_full, y_full, dates_full = load_data(ticker, start_date, end_date)
    if X_full is None:
        print(f"[WARN] Not enough data for {ticker}.")
        return None

    start_time = time.time()
    start_idx = 0

    results = []
    while True:
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size)
        if window_data[0] is None:
            print(f"[INFO] No more windows for {ticker}.")
            break
        X_window, y_window, dates_window = window_data

        for i in range(2, len(X_window)):
            sma_pred = sma_prediction(X_window[:i])
            ema_pred = ema_prediction(X_window[:i])
            lr_pred = linear_regression_prediction(X_window[:i])
            actual = y_window[i]
            date = dates_window[i]

            results.append({
                'Date': date,
                'Actual': actual,
                'SMA_Predicted': sma_pred,
                'EMA_Predicted': ema_pred,
                'LR_Predicted': lr_pred,
            })

        start_idx += step_size

    df_results = pd.DataFrame(results)
    df_results['SMA_Absolute_Error'] = np.abs(df_results['Actual'] - df_results['SMA_Predicted'])
    df_results['EMA_Absolute_Error'] = np.abs(df_results['Actual'] - df_results['EMA_Predicted'])
    df_results['LR_Absolute_Error'] = np.abs(df_results['Actual'] - df_results['LR_Predicted'])
    df_results.to_csv(f"baseline_results/{ticker}_results.csv", index=False)

    sma_mape = mean_absolute_percentage_error(df_results['Actual'], df_results['SMA_Predicted']) * 100.0
    ema_mape = mean_absolute_percentage_error(df_results['Actual'], df_results['EMA_Predicted']) * 100.0
    lr_mape = mean_absolute_percentage_error(df_results['Actual'], df_results['LR_Predicted']) * 100.0

    print(f"[INFO] {ticker}: SMA MAPE: {sma_mape:.2f}%, EMA MAPE: {ema_mape:.2f}%, LR MAPE: {lr_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] {ticker}: Elapsed time: {elapsed:.2f}s")

    return {
        'Stock': ticker,
        'SMA_MAPE': sma_mape,
        'EMA_MAPE': ema_mape,
        'LR_MAPE': lr_mape
    }

if __name__ == "__main__":
    all_results = []
    for stock in stocks:
        result = rolling_predictions(stock, window_size, step_size)
        if result:
            all_results.append(result)

    results_df = pd.DataFrame(all_results)
    results_df.to_csv('stock_mapes_combined_results.csv', index=False)
    print("[INFO] All results saved to stock_mapes_combined_results.csv")


[*********************100%%**********************]  1 of 1 completed
[INFO] No more windows for COST.
[INFO] COST: SMA MAPE: 1.68%, EMA MAPE: 11.26%, LR MAPE: 1.93%
[INFO] COST: Elapsed time: 0.27s
[*********************100%%**********************]  1 of 1 completed
[INFO] No more windows for UL.
[INFO] UL: SMA MAPE: 1.39%, EMA MAPE: 11.07%, LR MAPE: 1.73%
[INFO] UL: Elapsed time: 0.25s
[*********************100%%**********************]  1 of 1 completed
[INFO] No more windows for AMGN.
[INFO] AMGN: SMA MAPE: 1.67%, EMA MAPE: 11.13%, LR MAPE: 1.99%
[INFO] AMGN: Elapsed time: 0.25s
[*********************100%%**********************]  1 of 1 completed
[INFO] No more windows for UNH.
[INFO] UNH: SMA MAPE: 1.83%, EMA MAPE: 11.21%, LR MAPE: 2.18%
[INFO] UNH: Elapsed time: 0.26s
[*********************100%%**********************]  1 of 1 completed
[INFO] No more windows for WAT.
[INFO] WAT: SMA MAPE: 2.18%, EMA MAPE: 11.15%, LR MAPE: 2.64%
[INFO] WAT: Elapsed time: 0.26s
[*********************